In [92]:
#***************** SCRIPT GLOBAL ***********************#
#libreias
import pandas as pd

## NODOS 
# PROVINCIAS
P = ["AZUAY", "BOLIVAR", "CAÑAR", "CARCHI", "CHIMBORAZO", "COTOPAXI", "EL ORO", "ESMERALDAS", "GALAPAGOS", "GUAYAS", "LOS RIOS", "MANABI", "MORONA SANTIAGO", "NAPO", "ORELLANA", "PASTAZA", "PICHINCHA", "SANTA ELENA", "SANTO DOMINGO DE LOS TSACHILAS","SUCUMBIOS", "TUNGURAHUA", "ZAMORA CHINCHIPE"]
# PROVINCIAS Y CANTONES DICCIONARIO DE LISTA DE CANTONES PARA CADA PROVINICA 
Prov_Can = dict()
for p in P:
    Prov_Can[p] = list(pd.read_excel(p+".xlsx", sheet_name='Frecuencia')["Nom_Can"])


In [1]:
#***************** SCRIPT GLOBAL ***********************#
#libreias
import pandas as pd

## NODOS 
# PROVINCIAS
P = ["AZUAY", "LOJA", "IMBABURA", "BOLIVAR", "CAÑAR", "CARCHI", "CHIMBORAZO", "COTOPAXI", "EL ORO", "ESMERALDAS", "GALAPAGOS", "GUAYAS", "LOS RIOS", "MANABI", "MORONA SANTIAGO", "NAPO", "ORELLANA", "PASTAZA", "PICHINCHA", "SANTA ELENA", "SANTO DOMINGO DE LOS TSACHILAS","SUCUMBIOS", "TUNGURAHUA", "ZAMORA CHINCHIPE"]
# PROVINCIAS Y CANTONES DICCIONARIO DE LISTA DE CANTONES PARA CADA PROVINICA 

### Creacion de Parametros para los modelos de cada provincia.
## Tasa de Población
import pandas as pd
from gurobipy import * # Libreria de Solver Gurobi 
import numpy as np # Libreria para Matrices
import warnings

## Tasa de Poblacion
T1 = 8/100000 
T2 = 316 
## Modelo para cada provincia lee cada excel 

for p in P:
    ## Poblacion 
    Cantones = list(pd.read_excel(p +".xlsx", sheet_name='Frecuencia')["Nom_Can"])
    Poblacion = dict()
    i = 0 # Contador cantones
    for c in Cantones: 
        Poblacion[c] = list(pd.read_excel( p +".xlsx", sheet_name='Frecuencia')["Poblacion"])[i]
        i=i+1
    ## Demanda
    df = pd.DataFrame({'Name': ["PERSONAS Y GARANTIAS","DELINCUENCIA ORGANIZADA","ADMINISTRACION PUBLICA","SOLUCIONES RAPIDAS","VIOLENCIA DE GENERO","PATRIMONIO CIUDADANO","FE  PUBLICA","ACCIDENTES DE TRANSITO","MULTICOMPETENTE"]})
    Fiscalias = list(df["Name"]) # Nombre de Fiscalías
    Grupos = ["Grupo1","Grupo2", "Grupo3" ]
    Demanda = tupledict({})
    Nombres_Y = tuplelist([])
    Nombres_X = tuplelist([])
    for c in Cantones:
        df1 = pd.read_excel( p +".xlsx", sheet_name=c)
        for g in Grupos: 
            j = 0 # Contador Fiscalías
            for f in Fiscalias:
                Demanda[c,g,Fiscalias[j]] = list(df1[g])[j]
                Nombres_Y.append((c,Fiscalias[j]))
                Nombres_X.append((c,g,Fiscalias[j]))
                j=j+1

    Nombres_X = list(set(Nombres_X))
    Nombres_Y =list(set(Nombres_Y))
    ## Carga Laboral
    Carga = tupledict({})
    df1 = pd.read_excel( p +".xlsx", sheet_name= 'Frecuencia')

    i = 0      # Contador de Cantones
    for c in Cantones:
        for f in Fiscalias:
            if f == 'FISCALIA DE SOLUCIONES RAPIDAS':
                Carga[c,f] = round(list(df1["TASA_SR"])[i])
                i=i+1
            else:
                Carga[c,f] = 316 

    ## Multicompetente 
    Multi = dict() 
    i = 0    # Contador de Cantones
    for c in Cantones:
        Multi[c] = list(df1["MULTICOMPETENTE"])[i]
        i=i+1

    de =[] # Demandas pronosticadas total 
    C1 = []
    C2 = []
    for c in Cantones:
        C2.append(c)
        t = pd.read_excel( p +".xlsx", sheet_name=c)
        de.append(t["Grupo1"].sum()+ t["Grupo2"].sum() + t["Grupo3"].sum())
        if t["Grupo1"].sum()+ t["Grupo2"].sum() + t["Grupo3"].sum() <= 158:
            C1.append(c)
    print(C1)
    
    for c in C1:
        C2.remove(c)
    print(C2)
    ## *********************** CREACION DEL MODELO ***********************************

    ## Creamos el CONSTRUCTOR DEL MODELO 
    m = Model('Modelo_FGE')

    ## Creamos las Variables
    X = m.addVars(Nombres_X , name="", vtype=GRB.INTEGER) 
    Y = m.addVars(Nombres_Y , name="Fiscales_Necesarios", vtype=GRB.INTEGER)

    # Creacion de la FUNCIÓN OBJETIVO
    m.setObjective((Y.sum(Cantones,Fiscalias)) , GRB.MINIMIZE)

    ## Creacion de las Restricciones 
    # Restricción 1
    for c in C2:
        if Multi[c] == 0:
            for f in Fiscalias:
                m.addConstr((X.sum(c,Grupos,f) <= Y[c,f] * Carga[c,f]) , "tasa de carga")
        elif Multi[c] == 1:
            m.addConstr((X.sum(c, Grupos,Fiscalias) <= Y.sum(c,Fiscalias)*T2 ) , "tasa de carga_{}".format(f))

    # Restriccion 2      
    for f in Fiscalias:
        for c in C2:
            for g in Grupos:
                m.addConstr((X[c,g,f] == Demanda[c,g,f]), "demandadelitos")

    # Restriccion 3
    #for c in C2:
     #   m.addConstr(Y.sum(c,Fiscalias) >= Poblacion[c]* T1, "Se cumpla tasa T1")
    
    suma = 0
    for c in C1:
        suma = suma + Poblacion[c]
    
    m.addConstr(Y.sum(C1,Fiscalias) >= suma * T1, "Se cumpla tasa T1")
    m.addConstr((X.sum(C1, Grupos,Fiscalias) <= Y.sum(C1,Fiscalias)*T2 ), "Se cumpla tasa T1")
    

    m.optimize()

    warnings.simplefilter("ignore")
    ## Tabla de Resultados 1 
    Tabla1 = pd.DataFrame({'Name': ["PERSONAS Y GARANTIAS","DELINCUENCIA ORGANIZADA","ADMINISTRACION PUBLICA","SOLUCIONES RAPIDAS","VIOLENCIA DE GENERO","PATRIMONIO CIUDADANO","FE  PUBLICA","ACCIDENTES DE TRANSITO","MULTICOMPETENTE"]})
    for c in Cantones:
        Res = []
        for f in Fiscalias:
            Res.append(Y[c,f].x) 
        Tabla1[c] = Res

    for c in Cantones:
        if Multi[c] == 0:
            Tabla1[c][1] = Tabla1[c][1]+Tabla1[c][8]
            Tabla1[c][8] = 0
        elif Multi[c] == 1:
            suma = 0 
            for j in range(len(list(Tabla1[c]))):
                suma = suma + Tabla1[c][j]
                Tabla1[c][j] = 0
            Tabla1[c][8] = suma

    print(Tabla1)

    ## Tabla de Resultados 2 
    Tabla2 = pd.DataFrame({'Cantones': Cantones})
    Fo = []
    for c in Cantones:
        Fo.append(Tabla1[c].sum())

    Tabla2["Poblacion"] = list(pd.read_excel( p +".xlsx", sheet_name= 'Frecuencia')["Poblacion"])
    Tabla2["Delitos"] = de
    Tabla2["Fiscales Actuales"] = list(pd.read_excel( p +".xlsx", sheet_name= 'Frecuencia')["Fisc_Actuales"])
    Tabla2["Fiscales Óptimos"] = Fo
    Tabla2["Déficit"] = Tabla2["Fiscales Óptimos"]-Tabla2["Fiscales Actuales"] 
    Tabla2["Tasa Actual"] = (Tabla2["Fiscales Actuales"] * 100000)/Tabla2["Poblacion"]
    Tabla2["Tasa Óptima"] = (Tabla2["Fiscales Óptimos"] * 100000)/Tabla2["Poblacion"]
    print(Tabla2)

    writer = pd.ExcelWriter('Resultados_' + p + '.xlsx')
    Tabla1.to_excel(writer, sheet_name='Fiscalias', index=False)
    Tabla2.to_excel(writer, sheet_name='Resumen', index=False)
    writer.save()
    writer.close()


['SAN FERNANDO', 'SEVILLA DE ORO', 'GUACHAPALA', 'EL PAN', 'ONA']
['CUENCA', 'GUALACEO', 'CAMILO PONCE ENRIQUEZ', 'SANTA ISABEL', 'PAUTE', 'SIGSIG', 'GIRON', 'NABON', 'PUCARA', 'CHORDELEG']

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2022-02-18
Using license file C:\Users\HP\gurobi.lic
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 290 rows, 540 columns and 855 nonzeros
Model fingerprint: 0x107adc31
Variable types: 0 continuous, 540 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+03]
Found heuristic solution: objective 59.0000000
Presolve removed 290 rows and 540 columns
Presolve time: 0.27s
Presolve: All rows and columns removed

Explored 0 nodes (0 sim

[]
['IBARRA', 'OTAVALO', 'ANTONIO ANTE', 'COTACACHI', 'SAN MIGUEL DE URCUQUI', 'PIMAMPIRO']
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 178 rows, 216 columns and 378 nonzeros
Model fingerprint: 0xe0c0bfcd
Variable types: 0 continuous, 216 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+03]
Found heuristic solution: objective 35.0000000
Presolve removed 178 rows and 216 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 35 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.500000000000e+01, best bound 3.500000000000e+01, gap 0.0000%
                      Name  IBARRA  OTAVALO  ANTONIO ANTE  COTACACHI  \
0 

['SAN PEDRO DE HUACA']
['TULCAN', 'MONTUFAR', 'BOLIVAR', 'ESPEJO', 'MIRA']
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 150 rows, 216 columns and 360 nonzeros
Model fingerprint: 0x78c860b6
Variable types: 0 continuous, 216 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e-01, 6e+02]
Found heuristic solution: objective 20.0000000
Presolve removed 150 rows and 216 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 20 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.000000000000e+01, best bound 2.000000000000e+01, gap 0.0000%
                      Name  TULCAN  MONTUFAR  BOLIVAR  ESPEJO  MIRA  \
0     PERSONAS Y GAR

['MARCABELI', 'LAS LAJAS', 'ATAHUALPA', 'CHILLA']
['MACHALA', 'PASAJE', 'SANTA ROSA', 'HUAQUILLAS', 'EL GUABO', 'ARENILLAS', 'PINAS', 'PORTOVELO', 'ZARUMA', 'BALSAS']
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 314 rows, 504 columns and 810 nonzeros
Model fingerprint: 0xfd83f0b2
Variable types: 0 continuous, 504 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+03]
Found heuristic solution: objective 85.0000000
Presolve removed 314 rows and 504 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 85 

Optimal solution found (tolerance 1.00e-04)
Best objective 8.500000000000e+01, best bound 8.500000000000e+01, gap 0.0000%

Found heuristic solution: objective 6.0000000
Presolve removed 58 rows and 108 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 6 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.000000000000e+00, best bound 6.000000000000e+00, gap 0.0000%
                      Name  SANTA CRUZ  SAN CRISTOBAL  ISABELA
0     PERSONAS Y GARANTIAS         0.0            0.0      0.0
1  DELINCUENCIA ORGANIZADA         0.0            0.0      0.0
2   ADMINISTRACION PUBLICA         0.0            0.0      0.0
3       SOLUCIONES RAPIDAS         0.0            0.0      0.0
4      VIOLENCIA DE GENERO         0.0            0.0      0.0
5     PATRIMONIO CIUDADANO         0.0            0.0      0.0
6              FE  PUBLICA         0.0            0.0      0.0
7   ACCIDENTES DE TRANSITO         0.0            0.0      0.0
8          MULTICOMPETENTE         3.

['BUENA FE']
['QUEVEDO', 'BABAHOYO', 'VENTANAS', 'VINCES', 'VALENCIA', 'PUEBLO VIEJO', 'MOCACHE', 'MONTALVO', 'BABA', 'PALENQUE', 'URDANETA', 'QUINSALOMA']
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 354 rows, 468 columns and 801 nonzeros
Model fingerprint: 0xfb88736c
Variable types: 0 continuous, 468 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+03]
Found heuristic solution: objective 84.0000000
Presolve removed 354 rows and 468 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 84 

Optimal solution found (tolerance 1.00e-04)
Best objective 8.400000000000e+01, best bound 8.400000000000e+01, gap 0.0000%
          

['SAN JUAN BOSCO', 'PABLO VI']
['MORONA', 'SUCUA', 'GUALAQUIZA', 'PALORA ( METZERA )', 'SANTIAGO (MENDEZ)', 'LIMON INDANZA', 'TAISHA', 'HUAMBOYA', 'LOGRONO', 'TIWINTZA']
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 290 rows, 432 columns and 720 nonzeros
Model fingerprint: 0x42b323c4
Variable types: 0 continuous, 432 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e-01, 8e+02]
Found heuristic solution: objective 28.0000000
Presolve removed 290 rows and 432 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 28 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.800000000000e+01, best bound 2.800000000000e+01, gap 0.00

[]
['PASTAZA (PUYO)', 'MERA', 'ARAJUNO', 'SANTA CLARA']
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 122 rows, 144 columns and 252 nonzeros
Model fingerprint: 0xfa8d696f
Variable types: 0 continuous, 144 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+02]
Found heuristic solution: objective 18.0000000
Presolve removed 122 rows and 144 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 18 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.800000000000e+01, best bound 1.800000000000e+01, gap 0.0000%
                      Name  PASTAZA (PUYO)  MERA  ARAJUNO  SANTA CLARA
0     PERSONAS Y GARANTIAS             

Presolve removed 38 rows and 36 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 37 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.700000000000e+01, best bound 3.700000000000e+01, gap 0.0000%
                      Name  SANTO DOMINGO
0     PERSONAS Y GARANTIAS            6.0
1  DELINCUENCIA ORGANIZADA            2.0
2   ADMINISTRACION PUBLICA            1.0
3       SOLUCIONES RAPIDAS           16.0
4      VIOLENCIA DE GENERO            4.0
5     PATRIMONIO CIUDADANO            2.0
6              FE  PUBLICA            1.0
7   ACCIDENTES DE TRANSITO            5.0
8          MULTICOMPETENTE            0.0
        Cantones  Poblacion  Delitos  Fiscales Actuales  Fiscales Óptimos  \
0  SANTO DOMINGO     471132    10391                 31              37.0   

   Déficit  Tasa Actual  Tasa Óptima  
0      6.0     6.579897     7.853425

['YACUAMBI']
['ZAMORA', 'YANTZAZA', 'EL PANGUI', 'CHINCHIPE', 'CENTINELA DEL CONDOR', 'PALANDA', 'PAQUISHA', 'NANGARITZA']
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 234 rows, 324 columns and 549 nonzeros
Model fingerprint: 0x2d90bbc7
Variable types: 0 continuous, 324 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e-01, 1e+02]
Found heuristic solution: objective 19.0000000
Presolve removed 234 rows and 324 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 19 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.900000000000e+01, best bound 1.900000000000e+01, gap 0.0000%
                      Name  ZAMORA  YANTZAZ

In [4]:
P = ["AZUAY", "LOJA","IMBABURA", "BOLIVAR", "CAÑAR", "CARCHI", "CHIMBORAZO", "COTOPAXI", "EL ORO", "ESMERALDAS", "GALAPAGOS", "GUAYAS", "LOS RIOS", "MANABI", "MORONA SANTIAGO", "NAPO", "ORELLANA", "PASTAZA", "PICHINCHA", "SANTA ELENA", "SANTO DOMINGO DE LOS TSACHILAS","SUCUMBIOS", "TUNGURAHUA", "ZAMORA CHINCHIPE"]
print(len(P))

24


In [ ]:
    suma=0
    for c in C1:
        suma = suma + Poblacion[c]
    
    m.addConstr(X.sum(C1, Grupos,Fiscalias) >=  Y.sum(C1,Fiscalias) * suma)